# Tests Numpy

## Format JSON

In [1]:
import numpy as np
from pprint import pprint
from json_ntv import Ntv
from datetime import date
import pandas as pd
from shapely.geometry import Point
import ntv_pandas as npd
from numpy_ntv_connector import read_json, to_json, to_json_tab, read_json_tab



### Simple Ndarray (type 'ndarray')
This type represents a multi-dimensional array. This representation is lossless (if type is included).

In [2]:
ex = np.arange(1, 7).reshape((2, 3))

print("example (with dtype and without) :\n")
print(to_json(ex))
print(to_json(ex, dtype=False))

print("\nreversibility :\n")
ex2 = read_json(to_json(ex))
print(np.array_equal(ex2, ex))

example (with dtype and without) :

{':ndarray': ['int32', [[1, 2, 3], [4, 5, 6]]]}
{':ndarray': [[1, 2, 3], [4, 5, 6]]}

reversibility :

True


### Axed Ndarray (type 'xndarray')
This type is associated to a Ndarray with additional data (axes names, axes variables and metadata).

    - axes names : list of name of each axis
    - axes variables : list of 1-dimensional ndarray 
    - non-axes variables : N-dimensional xndarray where axes names are included in the axes names of the axed Ndarrray)
    - metadata : dict
    
Example :

```
 {'axes': ['x', 'y', 'z', 'option']
  'axesvars': [['x1', 'x2'],['y1', 'y2', 'y3'], ['z1', 'z2'], [True, False]],
  'data': ['int32',
           [[[[1, 2], [3, 4]], [[5, 6], [7, 8]], [[9, 10], [11, 12]]],
            [[[13, 14], [15, 16]], [[17, 18], [19, 20]], [[21, 22], [23, 24]]]]]}}   
   'vars':[ {'axes': ['x', 'y'], 'data': [['x1y1', 'x1y2', 'x1y3'], ['x2y1', 'x2y2', 'x2y3']] }, 
            {'axes': 'int_opt', 'data': [0, 1]} ]
```

In [3]:
a = np.arange(1, 25).reshape((2, 3, 2, 2))
axes = [{'x': ['x1', 'x2']}, 
        {'y': ['y1', 'y2', 'y3']}, 
        {'z': ['z1', 'z2']}, 
        {'option': [True, False]}]

print("\nexample with axes :\n")
pprint(to_json(a, axes=axes), width=150)

print("\nreversibility :\n")
a2, axes2 = read_json(to_json(a, axes=axes))
print(np.array_equal(a2, a), axes2 == axes)


example with axes :

{':ndarray': {'axes': [{'x': ['x1', 'x2']}, {'y': ['y1', 'y2', 'y3']}, {'z': ['z1', 'z2']}, {'option': [True, False]}],
              'data': ['int32',
                       [[[[1, 2], [3, 4]], [[5, 6], [7, 8]], [[9, 10], [11, 12]]],
                        [[[13, 14], [15, 16]], [[17, 18], [19, 20]], [[21, 22], [23, 24]]]]]}}

reversibility :

True True


### Multi-dimensional format is compatible with Tabular format
The multi_dimensional representation can be translated into tabular representation.

In [4]:
print("example without axes :\n")
pprint(to_json_tab(ex))

print("\nexample with axes :\n")
pprint(to_json_tab(a, axes), width=150)

print("\nreversibility :\n")
a2, axes2 = read_json_tab(to_json_tab(a, axes))
print(np.array_equal(a2, a), Ntv.obj(axes2) == Ntv.obj(axes))

example without axes :

{'axe0': [[0, 1], [3]],
 'axe1': [[0, 1, 2], [1]],
 'value::int32': [1, 2, 3, 4, 5, 6]}

example with axes :

{'option': [[true, false], [1]],
 'value::int32': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
 'x': [["x1", "x2"], [12]],
 'y': [["y1", "y2", "y3"], [4]],
 'z': [["z1", "z2"], [2]]}

reversibility :

True True


### Multi-dimensional format is compatible with tabular tools (e.g. Pandas).

In [10]:
print('pandas DataFrame :')
ex_df = npd.read_json(to_json_tab(ex))
display(ex_df)

a_df = npd.read_json(to_json_tab(a, axes))
display(a_df)
print(a_df.dtypes)

print("\nreversibility :\n")
a3, axes3 = read_json_tab(a_df.npd.to_json(header=False, index=False))
print(np.array_equal(a3, a), Ntv.obj(axes3) == Ntv.obj(axes))

pandas DataFrame :


,axe0,axe1,value
0,0,0,1
1,0,1,2
2,0,2,3
3,1,0,4
4,1,1,5
5,1,2,6


,x,y,z,option,value
0,x1,y1,z1,True,1
1,x1,y1,z1,False,2
2,x1,y1,z2,True,3
3,x1,y1,z2,False,4
4,x1,y2,z1,True,5
5,x1,y2,z1,False,6
6,x1,y2,z2,True,7
7,x1,y2,z2,False,8
8,x1,y3,z1,True,9
9,x1,y3,z1,False,10


x         category
y         category
z         category
option    category
value        int32
dtype: object

reversibility :

True True


### Using in NTV data

In [6]:
nd  = np.array([1, 2, 3, 4, 5, 6]).reshape(3, 2)
nd1 = np.array([1, 2, 3, 4, 5, 6]).reshape(2, 3)
nd2 = np.array([1, 2, 3, 4]).reshape(2, 2)
nd3 = np.array([1, 2, 3, 4])
# simple
simple = Ntv.obj({'simple:ndarray': nd})
print(simple)
print(simple.to_obj(format='obj'))
# list
a_list = Ntv.obj({'list::ndarray': [nd1, nd2, nd3]})
print('\n', a_list)
print(a_list.to_obj(format='obj'))
# mixte
mixte = Ntv.obj({'mixte': [nd2, Point(1,2), pd.Series([1,2,3])]})
mixte_json = mixte.to_obj()
print('\n', mixte_json)
print(mixte.to_obj(format='obj'))

{"simple:ndarray": ["int32", [[1, 2], [3, 4], [5, 6]]]}
{'simple': array([[1, 2],
       [3, 4],
       [5, 6]])}

 {"list::ndarray": [["int32", [[1, 2, 3], [4, 5, 6]]], ["int32", [[1, 2], [3, 4]]], ["int32", [1, 2, 3, 4]]]}
{'list::ndarray': [array([[1, 2, 3],
       [4, 5, 6]]), array([[1, 2],
       [3, 4]]), array([1, 2, 3, 4])]}

 {'mixte': {':ndarray': ['int32', [[1, 2], [3, 4]]], ':point': [1.0, 2.0], ':field': [1, 2, 3]}}
{'mixte': [array([[1, 2],
       [3, 4]]), <POINT (1 2)>, 0    1
1    2
2    3
dtype: int64]}


### Include Ndarray in other objects

In [7]:
sr = pd.Series([1, 2, np.array([1, 2, 3, 4])])
mixin = Ntv.obj({'mixin:field': sr})
print(mixin)


{"mixin:field": [1, 2, {":ndarray": ["int32", [1, 2, 3, 4]]}]}


## Example unit and quantity
- 'unit' is a specif type
- two options available for quantities:
    - option 1 : add specific types including unit
    - option 2 : add unit as type extension for existing types
    - option 3 : including unit in the name
    
Option 2 can be extended to other usages. For example:

```
    {"comment:string[fr]": "Paris est une belle ville"}
```
Option 2 and Option 1 are compatible with NTV struture. For example 
```
    {"list_of_ndarray::ndarray[kg]: { "array1": [[1, 2], [3, 4]], "array2": [[5, 6], [7, 8]]}}
```

In [11]:
# example Unit
{'mass:unit': 'kg'}
# example Quantity - option 1 (specific type)
{'ex_simple:qvalue':     [0.47, 'm / s']}
{'ex_simple_typ:qvalue': [['float64', 0.47], 'm / s']}
{'ex_array:qarray':      [[2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ], 'm / s']}
{'ex_array_typ:qarray':  [['float64', [2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ]], 'm / s']}
# example Quantity - option 2 (type extension)
{'ex_simple:[m / s]':            0.47}
{'ex_simple_typ:float64[m / s]': 0.47}
{'ex_array:ndarray[m / s]':      [2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ]}
{'ex_array_typ:ndarray[m / s]':  ['float64', [2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ]]}
# example Quantity - option 3 (name extension)
{'ex_simple-m / s':             0.47}
{'ex_simple_typ-m / s:float64': 0.47}
{'ex_array-m / s:ndarray':      [2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ]}
{'ex_array_typ-m / s:ndarray':  ['float64', [2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ]]}


{'ex_array_typ-m / s:ndarray': ['float64',
  [2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]]}

## Example table

In [9]:


quantity = {'simple:quantity':     [0.47, 'm / s']}
quantity = {'simple_typ:quantity': [['float64', 0.47], 'm / s']}

quantity = {'array:quantity':      [[2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ], 'm / s']}
quantity = {'array_typ:quantity':  [['float64', [2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ]], 'm / s']}

